## Wind Speed Display Application

The intent of this application is to give an overview of how advanced energy companies can use AWS cloud to build applications that can be useful in the energy world. This solution was easy to develop, but it would be more useful to iterate to an ideal solution than to try and get a complete solution all at once. Below is a quick iteration cycle about how I went about both learning about AWS and developing this solution. The final application can be found [here](http://dash-trail-env1.dmmtiygmw2.us-east-1.elasticbeanstalk.com/). 

Iteration | Desc | AWS service
--- | --- | ---
1 | run dash script locally | none
2 | run dyanamodb locally | dyanmodb on docker, IAM
3 | create and connect dyanamodb | dynamodb, IAM
4 | create a fake Sensor and run ec2| ec2, IAM
5 | change the dash script to run on beanstack| beanstack, IAM

By following an iterative approch, you can bring in different solutions in a progressive manner into your application stack. This is more managable than moving your complete platform into the cloud all at once. 

Below I will try to outline the major steps in creating this prototype. 

### Import the required libraries 
boto3 is the python client for AWS. 

In [1]:
import plotly.plotly as py
from plotly.graph_objs import *
from scipy.optimize import curve_fit
import numpy as np
import pandas as pd
from scipy.stats import norm
import plotly.figure_factory as ff
import math
from scipy.stats import skewnorm
import datetime as dt
import boto3
from decimal import Decimal
from boto3.dynamodb.conditions import Key, Attr

### Resource connection
Below we are going to initiate a resource from boto3. If you are using a local docker container, you need to provide the local docker endpoint url. --endpoint-url http://localhost:8000 

In [2]:
dynamodb = boto3.resource('dynamodb', region_name='us-east-1') 

### Getting data from the database 

Below is a sample application on how to get the data from the database. 

In [ ]:
def getMetaData(inputTag):
    metaWeatherDatatable = dynamodb.Table('metaWeatherDatatable')
    response = metaWeatherDatatable.get_item(
    Key={
        'tag':inputTag
    }
    )    
    return response['Item']


### Delete and insert data 
Below is a sample application on how we can insert data into the database. 

In [ ]:
def updateMetaData(inputData):
    # TODO: Need to update the meta data only when the date is older than update date 
    metaWeatherDatatable = dynamodb.Table('metaWeatherDatatable')
    response=metaWeatherDatatable.query(
        KeyConditionExpression = Key('tag').eq(inputData['tag'])
    )
    try:        
        data = response['Items'][0]
    except:
        data = None
    if data:        
        metaWeatherDatatable.delete_item(
        Key={
            'tag':inputData['tag']        
        }
        )
        metaWeatherDatatable.put_item(
            Item=inputData
        )
        finalResponse = 'Updated Data'
    else:
        print("noData")
        finalResponse = metaWeatherDatatable.put_item(Item=inputData)
    return finalResponse


In [ ]:
def putIntialData():
        timestamp = '2019-04-08 12:12:00'
        data = [
        {
            'tag':'Speed',
            'timestamp':timestamp,
            'value':Decimal(12.0)
        },
        {
            'tag':'SpeedError',
            'timestamp':timestamp,
            'value':Decimal(2.0)
        },
        {
            'tag':'Direction',
            'timestamp':timestamp,
            'value':Decimal(40)
        }
        
        ]


### Fake sensor

This is the fake sensor script. It's straightforward: it uses numpy and random number generation to essentially create a bunch of data points for the database. This script is eventually run on the ec2 server continiously so that we can create a bunch of fake data from an imaginary sensor.  

In [ ]:
def getSensorData():    
    now = dt.datetime.now()
    sec = now.second
    minute = now.minute
    hour = now.hour
    stTimestamp = dt.datetime(now.year, now.month, now.day, hour,minute,sec)
    try:
        prevVal = getMetaData('Speed')['value'] #starting wind speed to seed the random function     
    except:
        prevVal=20.0
    try:
        prevOrientation = getMetaData('Direction')['value'] #starting orentation of the wind direction
    except:
        prevOrientation = 50
    if(round(prevVal) > 45):
        prevVal = int(math.floor(prevVal))
    elif(round(prevVal) < 10):
        prevVal = int(math.ceil(prevVal))
    else:
        prevVal = int(round(prevVal))


    Speed=np.random.normal(prevVal, 2, 1)[0]
    SpeedError = np.random.normal(round(prevVal/10), 1)
    timestamp = stTimestamp.strftime("%Y-%m-%d %H:%M:%S")

    if(minute % 5 == 0):
        Direction = np.random.uniform(prevOrientation-50,
                                                prevOrientation+50)
    else:
        Direction = np.random.uniform(prevOrientation-5,
                                                prevOrientation+5)
    data = [
        {
            'tag':'Speed',
            'timestamp':timestamp,
            'value':Decimal(str(Speed))
        },
        {
            'tag':'SpeedError',
            'timestamp':timestamp,
            'value':Decimal(str(SpeedError))
        },
        {
            'tag':'Direction',
            'timestamp':timestamp,
            'value':Decimal(str(Direction))
        }
        
    ]
    return data


### Create tables
Below is how we can create tables into the database.

In [ ]:
weatherDatatable=dynamodb.create_table(
    TableName='weatherData',
    KeySchema=[
        {
            'AttributeName': 'tag',
            'KeyType': 'HASH'
        },
        {
            'AttributeName': 'timestamp',
            'KeyType': 'RANGE'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'tag',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'timestamp',
            'AttributeType': 'S'
        },
    ],
    
        ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)




In [ ]:
metaWeatherDatatable=dynamodb.create_table(
    TableName='metaWeatherDatatable',
    KeySchema=[
        {
            'AttributeName': 'tag',
            'KeyType': 'HASH'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'tag',
            'AttributeType': 'S'
        }
    ],
    
        ProvisionedThroughput={
        'ReadCapacityUnits': 2,
        'WriteCapacityUnits': 2
    }
)

### Create high-level objects
Below is how we can create a high-level object. Once the object is created we can utilize it to perform various operations. 

In [3]:
weatherDatatable = dynamodb.Table('weatherData')
weatherDatatable.meta.client.get_waiter('table_exists').wait(TableName='weatherData')
print(weatherDatatable.item_count)

1150


In [4]:
print(weatherDatatable.creation_date_time)

2019-04-09 18:27:27.310000-04:00


Below is a sample response for getting data. 

In [5]:
response=weatherDatatable.get_item(
Key={
    'tag':'windSpeed',
    'timestamp':'2019-04-07 19:50:00'
}
)
print(response)

{'ResponseMetadata': {'RequestId': '9O6R06ERHL4QA9N8IKOCOTSN27VV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Thu, 11 Apr 2019 00:48:09 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': '9O6R06ERHL4QA9N8IKOCOTSN27VV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}, 'RetryAttempts': 0}}


Below is a sample insert statement into the datbase. 

In [ ]:
weatherDatatable.put_item(Item=dbRow)

In [ ]:
weatherDatatable.delete_item(
Key={
    'tag':'windDirection',
    'timestamp':'2019-04-08 04:40:00',        
}
)

Below is how we can get data based on key value.  

In [ ]:
def getallTagData(inputTag):
    weatherDatatable = dynamodb.Table('weatherData')
    response=weatherDatatable.query(
        KeyConditionExpression =Key('tag').eq(inputTag)
    )
    items = response['Items']
    return items